In [0]:
print("please connect")

In [0]:
customer_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", "/Volumes/workspace/default/databricks-1/streaming/customer_autoloader/_schema")
        .load("/Volumes/workspace/default/databricks-1/streaming/customer_autoloader/")
)

In [0]:
customer_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", "/Volumes/workspace/default/databricks-1/customer_autoloader/_schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .load("/Volumes/workspace/default/databricks-1/customer_autoloader/")
)

In [0]:
customer_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", "/Volumes/workspace/default/databricks-1/customer_autoloader/_schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("cloudFiles.schemaHints","date_of_birth DATE, member_since DATE, created_timestamp TIMESTAMP")
        .load("/Volumes/workspace/default/databricks-1/customer_autoloader/")
)

In [0]:
from pyspark.sql.functions import current_timestamp, col
customer_transformed_df = (
        customer_df
            .withColumn("file_path", col("_metadata.file_path"))
            .withColumn("ingestion_date", current_timestamp())
)

In [0]:
%sql
drop table swathy_catalog.swathy_schema.customer_autoloader;
drop table swathy_catalog.swathy_schema.customer_autoloader_0;


In [0]:
streaming_query = (
    customer_transformed_df.writeStream
        .format("delta")
        .option("checkpointLocation","/Volumes/workspace/default/databricks-1/streaming/customer_autoloader/_checkpoints")
        .trigger(once=True)
        .toTable("swathy_catalog.swathy_schema.customer_autoloader_0")
)

In [0]:
display(spark.sql(
    """
    select * from swathy_catalog.swathy_schema.customer_autoloader
    """
))

In [0]:
streaming_query.stop()